In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
# Random
import random
random.seed = 42
# Thread management (sleep purposes)
import time
# Operating System management
import os
import selenium
# Selenium specific libraries
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

import pickle

In [ ]:
if not os.path.exists("portales/indomio"):
  os.makedirs("portales/indomio")

In [ ]:
configurations = {
    'url' : 'https://www.indomio.es/'}
service = Service(executable_path='chromedriver.exe')

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

driver.get(configurations["url"])
driver.maximize_window()
time.sleep(np.random.normal(2,0.3)) # segundos

driver.save_screenshot("portales/indomio/00-homepage.png")

In [ ]:
driver.find_element(by=By.ID, value="didomi-notice-agree-button").click()
time.sleep(np.random.normal(2,0.3)) # segundos
driver.save_screenshot("portales/fotocasa/01-homepage-no-cookies.png")

In [ ]:
campo_busqueda = driver.find_element(by=By.CLASS_NAME, value="nd-autocomplete__input")
time.sleep(np.random.normal(2,0.3)) # segundos
consulta = "valencia poblacion"
campo_busqueda.send_keys(consulta)

# Simular la presión de la tecla Enter para realizar la búsqueda (si es necesario)
campo_busqueda.send_keys(Keys.RETURN)

time.sleep(np.random.normal(2)) # segundos
driver.save_screenshot("portales/indomio/02-homepage-no-cookies.png")

In [ ]:
enlace = driver.find_element(by=By.CLASS_NAME, value= 'nd-accordion')
enlaces_hijos = enlace.find_elements(by=By.CLASS_NAME, value='im-zones__accordionLabelText')
distritos = []
for enlace_hijo in enlaces_hijos:
    distritos.append(enlace_hijo.get_attribute('title'))

In [ ]:
enlaces_hijos_2 = enlace.find_elements(by=By.CLASS_NAME, value='nd-accordion__subMenu')
barrios = []
for i in range(len(enlaces_hijos_2)):
    enlaces_hijos_3 = enlaces_hijos_2[i].find_elements(by=By.CLASS_NAME, value='nd-checkbox__text')
    barrios.append(enlaces_hijos_3)

In [ ]:
barriadas = []
for i in range(len(barrios)):
    for j in range(len(barrios[i])):
        barriadas.append(barrios[i][j].text)

In [ ]:
from unidecode import unidecode
for i in range(len(barriadas)):
    barriadas[i] = barriadas[i].replace(" ", "-").lower().replace("'","-").replace('*','-')
    barriadas[i] = unidecode(barriadas[i])

In [ ]:
with open('barrios_vlc.pkl', 'wb') as f:
    pickle.dump(barriadas, f)

In [ ]:
# Cargar el diccionario desde el archivo
with open('barrios_vlc.pkl', 'rb') as f:
    barridas = pickle.load(f)

In [ ]:
distritos = distritos[1:]

In [ ]:
# se toma el tamaño de barrios,ese es el numero de barriadas que tiene cada distrito
# se crea un diccionario que tiene como clave el distrito y como valor una lista de barriadas
distrito_barrios = {}
j = 0
for i in range(len(distritos)):
    distrito_barrios[distritos[i]] = barriadas[j:j+len(barrios[i])]

    j = j + len(barrios[i])

In [ ]:
def get_url(i,j):
    return f'https://www.indomio.es/venta-casas/valencia-capital/{barriadas[j]}/?pag={i}'
i = 0
j = 0
configurations = {
    'url': get_url(i,j),
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

response = requests.get(configurations['url'], headers={'User-Agent': configurations['user-agent']})

# Parsing the page
soup = BeautifulSoup(response.text, 'html.parser')
anuncios = soup.find_all('li', class_='nd-list__item in-reListItem')
links = []
titulos_casas = []
anuncios = 0

for j in range(len(barriadas)):
    i = 0
    ratio = 0
    print(f'Barriada: {barriadas[j]}')
    configurations['url'] = get_url(i, j)
    
    while i <= ratio:
        print("Página actual:", configurations['url'])  
        response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})
        
        if response.status_code == 200:
            # Parsear el contenido HTML
            soup = BeautifulSoup(response.text, 'html.parser')
            anuncios_header = soup.find('div', class_='in-realEstateListHeader__title').text
            matches = re.findall(r'\d+', anuncios_header)
            # Check if any matches were found
            if matches:
                # Extract the first match and convert it to an integer
                anuncios_number = int(matches[0])
                print("Number extracted:", anuncios_number)
            else:
                anuncios_number = 0
            ratio = anuncios_number / 25
            print(f'Ratio: {ratio}')
                                
            anuncios = soup.find_all('li', class_='nd-list__item in-reListItem')
            for anuncio in anuncios:
                enlace = anuncio.find('a').get('href')
                time.sleep(random.randint(1, 5) / 10)
                links.append(enlace)
                titulo = anuncio.find('a').get('title')
                time.sleep(random.randint(1, 5) / 10)
                titulos_casas.append(titulo)
                
            i += 1
            configurations['url'] = get_url(i, j)

        else:
            # Si la solicitud no fue exitosa, imprimir un mensaje de error y salir del bucle
            print("Error al obtener la página:", response.status_code)
            break

In [ ]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
unique_links = f7(links)
unique_titles = f7(titulos_casas)

In [ ]:
import pickle

with open('links.pkl', 'wb') as f:
    pickle.dump(unique_links, f)

In [ ]:
# Cargar el diccionario desde el archivo
with open('links.pkl', 'rb') as f:
    unique_links = pickle.load(f)

In [ ]:
diccionario_casas = {}
i = 0
# Iteramos sobre los enlaces únicos y los usamos como claves en el diccionario
for link in unique_links:

    # Creamos un diccionario interno con una clave 'i' y un valor None
    diccionario_casas[link] = {f'{i}': {'Precio':None,
                                        'Descripcion':None,
                                        'Otras_caractericas':None,
                                        'Caracteristicas':None,
                                        'Eficiencia_energetica':None}}
    i += 1 

In [ ]:
for link in unique_links:
    configurations = {
        'url': f'{link}',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})    
    soup = BeautifulSoup(response.text, 'html.parser')    

    descripcion_elem = soup.find('div', class_='in-readAll')
    descripcion = None if descripcion_elem is None else descripcion_elem.text
    caracteristicas_titulo = soup.find_all('dl', class_='in-realEstateFeatures__list')
    
    if len(caracteristicas_titulo) < 4:
        
        titulos = caracteristicas_titulo[0].find_all('dt', class_='in-realEstateFeatures__title')
        valores = caracteristicas_titulo[0].find_all('dd', class_='in-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[1].find('dd', class_='in-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[2].find_all('dt', class_='in-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[2].find_all('dd', class_='in-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]
        
        otr = None
    
    else:
        
        otros = caracteristicas_titulo[0].find_all('div', class_='nd-badge')
        otr = [valor.text for valor in otros]

        titulos = caracteristicas_titulo[1].find_all('dt', class_='in-realEstateFeatures__title')
        valores = caracteristicas_titulo[1].find_all('dd', class_='in-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[2].find('dd', class_='in-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[3].find_all('dt', class_='in-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[3].find_all('dd', class_='in-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]

    # Crear el diccionario para almacenar la información
    diccionario_casas[link] = {
        'Precio': precio,
        'Descripcion': descripcion,
        'Otras_caractericas': otr,
        'Caracteristicas': dict(zip(caracteristicas, values)),
        'Eficiencia_energetica': dict(zip(tit_ef, ef))
    }

In [ ]:
dict_iterator = iter(diccionario_casas.items())

# Get the first item
first_link, first_casa = next(dict_iterator)

# Print the details of the first house
print(f"Link: {first_link}")
print(f"Precio: {first_casa['Precio']}")
print(f"Descripcion: {first_casa['Descripcion']}")
print(f"Otras_caractericas: {first_casa['Otras_caractericas']}")
print(f"Caracteristicas: {first_casa['Caracteristicas']}")
print(f"Eficiencia_energetica: {first_casa['Eficiencia_energetica']}")

In [ ]:
next_link, next_casa = next(dict_iterator)
# Print the details of the first house
print(f"Link: {next_link}")
print(f"Precio: {next_casa['Precio']}")
print(f"Descripcion: {next_casa['Descripcion']}")
print(f"Otras_caractericas: {next_casa['Otras_caractericas']}")
print(f"Caracteristicas: {next_casa['Caracteristicas']}")
print(f"Eficiencia_energetica: {next_casa['Eficiencia_energetica']}")

In [ ]:
with open('scrap_info.pkl', 'wb') as f:
    pickle.dump(diccionario_casas, f)

In [ ]:
# Cargar el diccionario desde el archivo
with open('links.pkl', 'rb') as f:
    diccionario_casas = pickle.load(f)